In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as data
import yfinance as yf

In [ ]:
tart='2010-01-01'
end='2015-12-31'
#d=data.DataReader("TSLA","yahoo", tart, end)
df = yf.download('TSLA', start='2010-01-01', end='2023-03-23')
df.head()

#data
#d.head()

In [ ]:
df=df.reset_index()
df.head()
df.tail()


In [ ]:
#df=df.drop(['Date','Adj Close'],axis=0)
df=df.drop('Adj Close',axis=1)
df.head()

In [ ]:
plt.plot(df.Close)

In [ ]:
ma100=df.Close.rolling(100).mean()
print(ma100)
plt.plot(ma100,'r')
plt.plot(df.Close,'b')

In [ ]:
ma200=df.Close.rolling(200).mean()
print(ma200)
plt.plot(ma100,'r')
plt.plot(df.Close,'b')
plt.plot(ma200,'y')

In [ ]:
#data training
data_train=pd.DataFrame(df['Close'][0:2560])
data_test=pd.DataFrame(df['Close'][2560:3204])
print(data_train.shape)
print(data_test.shape)
data_train.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
data_traina=scaler.fit_transform(data_train)
data_traina

In [ ]:
x_train=[]
y_train=[]

for i in range(100,data_traina.shape[0]):
    x_train.append(data_traina[i-100:i])
    y_train.append(data_traina[i:0])

x_train,y_train=np.array(x_train),np.array(y_train)
    

In [ ]:
x_train.shape

In [ ]:
#ml model
from keras.layers import Dense,Dropout,LSTM
from keras.models import Sequential

model = Sequential([

    LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(units=60, return_sequences=True),
    Dropout(0.3),
    LSTM(units=70),
    Dropout(0.4),
    Dense(units=1)
])

In [ ]:
#model.compile(optimizer='adam',loss='mean_squared_error')
#model.fit(x_train,y_train,epochs=20)

In [ ]:
from keras.models import load_model

#saving the model 
#model.save('nn.h5')
#loading it
modeln=load_model('nn.h5')

In [ ]:
past100=data_train.tail(100)
final=past100.append(data_test,ignore_index=True)
final.head()

In [ ]:
inputd=scaler.fit_transform(final)
inputd

In [ ]:
inputd.shape

In [ ]:
x_test=[]
y_test=[]
for i in range(100,inputd.shape[0]):
    x_test.append(inputd[i-100:i])
    y_test.append(inputd[i,0])
x_test,y_test=np.array(x_test),np.array(y_test)
print(x_test.shape,y_test.shape)

In [ ]:
#Predictions

# y_predict=modeln.predict(x_test)

y_test
y=modeln.predict(x_test)
print(y)

In [ ]:
print(y.shape)
print(y)

In [ ]:
print (scaler.scale_)

scale=1/0.00267661
y_predict=y_predict*scale
y_test=y_test


In [ ]:
#plt.plot(y_predict,'r',label="predicted")
plt.plot(y_test*scale,'b',label="og")
plt.plot(y*scale*-18,'g',label="predicted")
plt.xlabel('time')
plt.ylabel('price')
plt.show()